# Taller 4

## Cargar CSV a MySQL con carga inicial e incremental

In [16]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

load_dotenv()  # Cargar variables de entorno desde el archivo .env

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

ddl = """
CREATE TABLE IF NOT EXISTS ventas (
    venta_id INT PRIMARY KEY AUTO_INCREMENT,
    cliente varchar(100),
    producto varchar(100),
    categoria varchar(100),
    precio_unitario decimal(10,2),
    unidades int,
    fecha DATE
);
"""

with engine.connect() as conn:
    conn.execute(text(ddl))

print("✅ Tabla creada exitosamente")


✅ Tabla creada exitosamente


In [17]:
#leer el archivo CSV
df = pd.read_csv('ventas.csv')

#limpieza
df['precio_unitario'] = pd.to_numeric(df['precio_unitario'], errors='coerce')
df['unidades'] = pd.to_numeric(df['unidades'], errors='coerce')
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce').dt.date

#carga inicial: remplazar contenido de la tabla
with engine.begin() as conn:
    conn.execute(text("TRUNCATE TABLE ventas"))
df.to_sql('ventas', con=engine, if_exists='append', index=False, method='multi', chunksize=1000)

print(f"Datos cargados en la tabla ventas. Total registros: {len(df)}")

Datos cargados en la tabla ventas. Total registros: 20


In [ ]:
from sqlalchemy import text

df_inc = pd.read_csv('ventas_incrementales.csv')
df_inc['precio_unitario'] = pd.to_numeric(df_inc['precio_unitario'], errors='coerce')
df_inc['unidades'] = pd.to_numeric(df_inc['unidades'], errors='coerce')
df_inc['fecha'] = pd.to_datetime(df_inc['fecha'], errors='coerce').dt.date

# canvertimos lista de diccionarios para ejecuciones masivas
records = df_inc.to_dict(orient='records')

upsert_sql = text("""
INSERT INTO ventas (venta_id, cliente, producto, categoria, precio_unitario, unidades, fecha)
VALUES (:venta_id, :cliente, :producto, :categoria, :precio_unitario, :unidades, :fecha)
ON DUPLICATE KEY UPDATE
    cliente = VALUES(cliente),
    producto = VALUES(producto),
    categoria = VALUES(categoria),
    precio_unitario = VALUES(precio_unitario),
    unidades = VALUES(unidades),
    fecha = VALUES(fecha);
""")
batch = 1000
with engine.begin() as conn:
    for i in range(0, len(records), batch):
        conn.execute(upsert_sql, records[i:i+batch])
print(f"Datos incrementales cargados. Registros procesados: {len(df_inc)}")                  

Datos incrementales cargados. Registros procesados: 4


In [19]:
# mostrar primeras y ultimas filas

all_rows = pd.read_sql("SELECT * FROM ventas", con=engine)
print(all_rows.head(5))
print(all_rows.tail(5))
print("total registros", len(all_rows))

# top productos por ventas (precio * unidades)
top5 = pd.read_sql("""
SELECT producto, SUM(precio_unitario * unidades) AS total
FROM ventas
GROUP BY producto
ORDER BY total DESC
LIMIT 5;
""", con=engine)

print("top 5 productos por ventas")
print(top5)

   venta_id cliente   producto    categoria  precio_unitario  unidades  \
0         1     Ana     Laptop  Electrónica           2500.0         1   
1         2    Luis   Teléfono  Electrónica           1200.0         2   
2         3   Cesar     Tablet  Electrónica            800.0         1   
3         4   María  Audífonos   Accesorios            150.0         3   
4         5   Pedro    Monitor  Electrónica            650.0         1   

        fecha  
0  2023-01-01  
1  2023-01-02  
2  2023-01-02  
3  2023-01-03  
4  2023-01-15  
    venta_id cliente producto    categoria  precio_unitario  unidades  \
18        19   Pedro  Monitor  Electrónica            610.0         1   
19        20   Laura  Teclado   Accesorios            105.0         1   
20        21    Nora   Cámara  Electrónica            350.0         1   
21        22  Andrés  Lámpara   Accesorios             40.0         2   
22        23     Leo  Lámpara   Accesorios             40.0         2   

         fecha  
18 